In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from preprocessing import preprocess

In [2]:
data = pd.read_csv("train.csv", index_col=0)

In [3]:
data.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312


In [4]:
data = preprocess(data)
data.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
0,Jansport,Leather,Medium,7.0,1,0,Tote,Black,11.611723,112.15875
1,Jansport,Canvas,Small,10.0,1,1,Messenger,Green,27.078537,68.88056
2,Under Armour,Leather,Small,2.0,1,0,Messenger,Red,16.643760,39.17320
3,Nike,Nylon,Small,8.0,1,0,Messenger,Green,12.937220,80.60793
4,Adidas,Canvas,Medium,1.0,1,1,Messenger,Green,17.749338,86.02312


In [5]:
numerical_columns = data.select_dtypes(include='number').columns
categorical_columns = data.select_dtypes(exclude='number').columns

In [6]:
print(f"{(len(numerical_columns) + len(categorical_columns))} = {len(data.columns)}")

10 = 10


In [7]:
data.columns

Index(['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price'],
      dtype='object')

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    OrdinalEncoder,
)
from sklearn.compose import ColumnTransformer

In [9]:
ohe_columns = [
    "Brand",
    "Material",
    "Size",
    #"Compartments",
    "Style",
    "Color"
]

columns_to_scale = [
    "Compartments",
    "Weight Capacity (kg)"
]

In [10]:
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
std = StandardScaler()

In [11]:
column_transformer = ColumnTransformer(
    transformers=[
        ("OneHotEncoding", ohe, ohe_columns),
    ],
    remainder="passthrough",
    n_jobs=-1,
    verbose=3,
    verbose_feature_names_out=False
).set_output(transform="pandas")

In [12]:
pipeline = Pipeline(
    [
        ("ColumnTransformer", column_transformer)
    ]
)

In [13]:
pipeline.fit_transform(data)

,Brand_Adidas,Brand_Inconnu,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Inconnu,Material_Leather,Material_Nylon,...,Color_Gray,Color_Green,Color_Inconnu,Color_Pink,Color_Red,Compartments,Laptop Compartment,Waterproof,Weight Capacity (kg),Price
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,1,0,11.611723,112.15875
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,10.0,1,1,27.078537,68.88056
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2.0,1,0,16.643760,39.17320
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,8.0,1,0,12.937220,80.60793
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1,1,17.749338,86.02312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0,0,12.730812,129.99749
299996,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0,1,26.633182,19.85819
299997,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,9.0,1,1,11.898250,111.41364
